In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
screen_name = 'ola_supports'
db = MongoClient().get_database('not-truncated')
convs = list(db.conversations.find({'screen_name': screen_name, 'topics': {'$ne': []}}))
print('There are total', len(convs), 'conversations.')

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused

In [ ]:
# flatten it

dataset = []
topics_to_ignore = ['undefined', 'poor_experience', 'random_query']
cant_coexist = [
    ('overcharged', 'payments'),
    ('cancellation_charged', 'overcharged'),
    ('driver_denied', 'cancellation_charged'),
]
# cant_coexist = []

multi_topic_convs = []

for c in convs:
    topics_here = [t for t in c['topics'] if t not in topics_to_ignore]
    
    # manually remove second topic from pairs if they co-exist
    for to_keep, to_discard in cant_coexist:
        if to_keep in topics_here and to_discard in topics_here:
            topics_here.remove(to_discard)
    
    
    for m in filter(lambda msg: msg['important'], c['messages']):
        multi_topic_convs.append({'message': m['processed_text'], 'topics': topics_here})
        for t in topics_here:
            dataset.append({'message': m['processed_text'].replace('\n', ''), 'topic': t})
        
multi_topic_df = pd.DataFrame(multi_topic_convs)
df = pd.DataFrame(dataset)
print(df.size, 'records')

df.head()

In [ ]:
# get topic counts

topic_counts = df['topic'].value_counts()
print(topic_counts)
print(topic_counts.sum())
topic_counts.plot.bar()

In [ ]:
# remove insignificant topics
cutoff_freq = int(0.1 * len(convs))  # 10 % is manually selected, coz it's near 50
print(cutoff_freq, 'is the cutoff frequency.')

selected_topics = topic_counts[topic_counts > cutoff_freq]
print('Selected topics are', selected_topics.index.values)

l1 = df.size
df = df.loc[df['topic'].isin(selected_topics.index.values)]
l2 = df.size
print(l1, 'records reduced to', l2, 'records.')

original_df = df.copy()
df['topic'].value_counts().plot.bar()

In [ ]:
# put topic_id on topics to make it a numerical value
df['topic_id'] = df['topic'].factorize()[0]

topic_id_df = df[['topic', 'topic_id']].drop_duplicates().sort_values('topic_id')
topic_to_id = dict(topic_id_df.values)
id_to_topic = dict(topic_id_df[['topic_id', 'topic']].values)

In [ ]:
# get features and labels using text vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8',
                       ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.message)
labels = df['topic_id']
features.shape, labels.shape

In [ ]:
# find most correlated topics
from sklearn.feature_selection import chi2

N = 3
for topic, topic_id in sorted(topic_to_id.items()):
    features_chi2 = chi2(features, labels == topic_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split()) == 1]
    bigrams = [v for v in feature_names if len(v.split()) == 2]
    trigrams = [v for v in feature_names if len(v.split()) == 3]
    print('For', topic)
    print('\t', 'Unigrams >', unigrams[-N:])
    print('\t', 'Bigrams >', bigrams[-N:])

In [ ]:
# make required imports for training on above dataset

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# for metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

# for undersampling or oversampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours as ENN

# for some vis
import seaborn as sns

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df['message'], df['topic_id'], random_state=0)

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8',
                       ngram_range=(1, 2), stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)

print(X_train_tfidf.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [ ]:
# some custom accuracy methods

def accuracy(clf):
    Y_pred = clf.predict(tfidf.transform(multi_topic_df['message']))
    total = sum(1 for a, p in zip(multi_topic_df['topics'], Y_pred) if id_to_topic[p] in a)
    return total / len(multi_topic_df)

In [ ]:
# try models one by one

def run_a_model(model, imb=None):
    print('Running for', model.__class__.__name__, 'with imbalance fixing by', imb.__name__ if imb else None)
    if not imb:
        clf = model.fit(X_train_tfidf, Y_train)
    else:
        X_train_res, Y_train_res = imb(random_state=0).fit_sample(X_train_tfidf, Y_train)
        clf = model.fit(X_train_res, Y_train_res)
    Y_pred = clf.predict(tfidf.transform(X_test))

    print('Actual accuracy :', accuracy(clf))
    print('Model Accuracy :', accuracy_score(Y_test, Y_pred))
    print('Recall :', recall_score(Y_test, Y_pred, average='micro'))
    print('Precision :', precision_score(Y_test, Y_pred, average='micro'))

    conf_mat = confusion_matrix(Y_test, Y_pred)
    n_topics = len(selected_topics)
    fig, ax = plt.subplots(figsize=(n_topics, n_topics))
    sns.heatmap(conf_mat, annot=True, fmt='d', 
                xticklabels=selected_topics.index.values,
                yticklabels=selected_topics.index.values)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()
    return clf

In [ ]:
# multinomial naive bayes
run_a_model(MultinomialNB())
run_a_model(MultinomialNB(), SMOTE)
run_a_model(MultinomialNB(), ENN)

# logistic regression
run_a_model(LogisticRegression(random_state=0))
run_a_model(LogisticRegression(random_state=0), SMOTE)
run_a_model(LogisticRegression(random_state=0), ENN)

# linear svc
run_a_model(LinearSVC())
run_a_model(LinearSVC(), SMOTE)
run_a_model(LinearSVC(), ENN)

# random forest classifier
run_a_model(RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0))
run_a_model(RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0), SMOTE)
run_a_model(RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0), ENN)